# DeconvoluteBloodEPIC

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)


Let's first import some packages:

In [17]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [18]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.DeconvoluteBloodEPIC)


class DeconvoluteBloodEPIC(DeconvolutionSingleCell):
    def __init__(self):
        super().__init__()



In [19]:
model = pya.models.DeconvoluteBloodEPIC()

## Define clock metadata

In [20]:
model.metadata["clock_name"] = 'deconvolutebloodepicbcell'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2024
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Ying, Kejun, et al. \"A unified framework for systematic curation and evaluation of aging biomarkers.\" Nature Aging (2025): 1-17."
model.metadata["doi"] = "https://doi.org/10.1038/s43587-025-00987-y"
model.metadata["research_only"] = None
model.metadata["notes"] = "Estimated proportion of b_cell from EPIC reference."


## Download clock dependencies

#### Download reference file

In [21]:
coeff_url = "https://raw.githubusercontent.com/bio-learn/biolearn/master/biolearn/data/EPIC_salas_18_reference.csv"
os.system(f"curl -L {coeff_url} -o EPIC_salas_18_reference.csv")


0

## Load features

#### From CSV file

In [22]:
import numpy as np
import pandas as pd

ref = pd.read_csv('EPIC_salas_18_reference.csv', index_col=0)
model.features = ref.index.astype(str).tolist()

ref_matrix = torch.tensor(ref.values, dtype=torch.float64)
pseudo_inv = torch.linalg.pinv(ref_matrix)

model.pseudo_inv = pseudo_inv
model.cell_index = 2
model.reference_values = torch.nanmean(ref_matrix, dim=1)


## Load weights into base model

#### From CSV file

In [23]:
# No linear base model; deconvolution logic in model forward


#### Linear model

In [24]:
model.base_model = None


## Load reference values

In [25]:
# reference_values already set above

## Load preprocess and postprocess objects

In [26]:
model.preprocess_name = "fill_with_reference_means"
model.preprocess_dependencies = None


In [27]:
model.postprocess_name = None
model.postprocess_dependencies = None


## Check all clock parameters

In [28]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': 'Ying, Kejun, et al. "A unified framework for systematic curation '
             'and evaluation of aging biomarkers." Nature Aging (2025): 1-17.',
 'clock_name': 'deconvolutebloodepicbcell',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1038/s43587-025-00987-y',
 'notes': 'Estimated proportion of b_cell from EPIC reference.',
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2024}
reference_values: [0.20518415451984126, 0.19970558314682532, 0.1024157725119047, 0.2008561107023811, 0.14881559691666668, 0.18691947926984118, 0.09904868163095237, 0.11866279766269845, 0.11413279887698413, 0.07614163463888889, 0.11287183193253969, 0.09002383478571424, 0.15332935412698398, 0.10906152481746033, 0.12821537873412694, 0.11714246567063487, 0.1349594052698413, 0.21058724882142854, 0.239

## Basic Test

In [29]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[0.0999],
        [0.0981],
        [0.0000],
        [0.0413],
        [0.2052],
        [0.0000],
        [0.0000],
        [0.0068],
        [0.0205],
        [0.0781]], dtype=torch.float64)

## Save torch model


In [30]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")


## Clear directory
<a id="10"></a>


In [31]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: EPIC_salas_18_reference.csv
